In [17]:
# Call assemblyai api to process audio

import assemblyai as aai
from datetime import timedelta, datetime, timezone
import os
from azure.storage.blob import BlobSasPermissions, BlobClient, generate_blob_sas


In [22]:
# Retrieve a read only SAS url for the blob
blob_client = BlobClient.from_blob_url("https://classroomtranscripts.blob.core.windows.net/uploads/SW Millikan Way 3.m4a")
blob_service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_STORAGE_CONNECTION_STRING"))
sas_expiry = datetime.now(timezone.utc) + timedelta(hours=1)
sas_start = datetime.now(timezone.utc) - timedelta(minutes=5)
blob_sas = generate_blob_sas(
    account_name=blob_client.account_name,
    container_name=blob_client.container_name,
    blob_name=blob_client.blob_name,
    permission=BlobSasPermissions(read=True),
    expiry=sas_expiry,
    start=sas_start,
    protocol="https",
    account_key=blob_service_client.credential.account_key,
) 
blob_sas

'st=2025-02-10T14%3A52%3A37Z&se=2025-02-10T15%3A57%3A37Z&sp=r&spr=https&sv=2025-01-05&sr=b&sig=EvA5JJG0PfSECAHKYZrdgQM2eS/QZX9ZB11FS8XdL5A%3D'

In [32]:
# Construct the SAS URL
blob_url = f"https://{blob_client.account_name}.blob.core.windows.net/{blob_client.container_name}/{blob_client.blob_name}"
sas_url = f"{blob_url}?{blob_sas}"
sas_url

'https://classroomtranscripts.blob.core.windows.net/uploads/SW Millikan Way 3.m4a?st=2025-02-10T14%3A52%3A37Z&se=2025-02-10T15%3A57%3A37Z&sp=r&spr=https&sv=2025-01-05&sr=b&sig=EvA5JJG0PfSECAHKYZrdgQM2eS/QZX9ZB11FS8XdL5A%3D'

In [34]:
# Format the blob sas url with escaped spaces
blob_sas_url = sas_url.replace(" ", "%20")
blob_sas_url
print(f"Blob SAS URL: {blob_sas_url}")


Blob SAS URL: https://classroomtranscripts.blob.core.windows.net/uploads/SW%20Millikan%20Way%203.m4a?st=2025-02-10T14%3A52%3A37Z&se=2025-02-10T15%3A57%3A37Z&sp=r&spr=https&sv=2025-01-05&sr=b&sig=EvA5JJG0PfSECAHKYZrdgQM2eS/QZX9ZB11FS8XdL5A%3D


In [ ]:
# Download the blob
blob_data = blob_client.download_blob().readall()
blob_data

In [36]:
aai.settings.api_key = os.getenv("ASSEMBLYAI_API_KEY")

transcriber = aai.Transcriber()

# transcript = transcriber.upload_file(
#     "https://classroomtranscripts.blob.core.windows.net/uploads/SW Millikan Way 3.m4a"
# )
transcript = transcriber.transcribe(blob_sas_url)
transcript

In [38]:
transcript.text

"First of all, she said this was the best PD that she's ever been to. So, Chad, really, like, over the top. She's effusive. And she was really grateful to Kim for reaching out, for offering to reach out to her essential resources and to be someone she could be in contact with. So in the interview, we explored the importance of having access to a community to ask questions. So I think. I know you guys are building that out, so I think that that is something that people are really looking for. Good. Yeah, that's valid. Yeah. She's very much alone in her school. That's the trend that I noticed here, was that remoteness of, hey, I'm the only one. It's not that I'm the only one in my district teaching biology. I'm the only one teaching science 6th grade through 12th grade. I'm the only one. And I would bet that would only serve. That. I don't imagine that would replace the desire to come back next year. I would think that would even. Yeah. Increase it. And that might be a third of the teach